# Using GMT from Julia ... and never come back

See the [manual](https://www.generic-mapping-tools.org/GMTjl_doc/) with lots of examples

In [1]:
# Start the GMT session
using GMT

Download a chunk of the earth_relief grid at 5 minutes grid spacing.
The first time it has to download the data so it may take a little.

In [ ]:
G = grdcut("@earth_relief_05m", region="IHO23");

In [ ]:
# Display some of the grid's metadata
show(G)

We can make crazy things with this grid object addressing to it with just its variable name, *G*,
as for example use it in a *calculator*

In [ ]:
# Get the array's minmax (like gmtinfo)
extrema(G)

In [ ]:
# Some dumb calculations
G2 = sqrt.(G .+ 10000) ./ 2;	# Does NOT make grid copies
setgrdminmax!(G2)				# Need to update z_min/max
extrema(G2)

OK, but let's go back to the bathymetry grid and make a quick but very nice image that uses the
GMT.jl guessing powers. The command automatically pick an apropriate projection, adds coastlines,
a colorbar and shows the result.

In [ ]:
imshow(G, proj=:guess, cmap=:geo, shade=true, coast=true, colorbar=true)

But if we want to add more elements to the figure and can't do it in a single shot. In that case
the `imshow` command is not the most appropriate. We will use `grdimge`, `meca` and `plot` modules to
add a focal mechanism and a volcano. We plot also the Maritime Boundaries extracted directly from a
zipped shapefile without even uncompress it.

In [ ]:
# Lay down the basemap that includes a shaded image, coastlines and a colorbar 
grdimage(G, proj=:guess, cmap=:geo, shade=true, coast=true, colorbar=true)

# Read and plot the EEZ from a low resolution zipped shapefile (no unzipping is needed)
eez =  gmtread("/vsizip/EEZ.shp.zip", ogr=true);
plot!(eez, lw=0.75, ls=:dash, lc=:blue)

# Create an example focal mechanism of a strike slip event
D = mat2ds([-42.4 10.7 0.0 0 90 0 5 -40 17],["Right Strike-slip"]);
gmtwrite("onemeca.dat", D)		# Due to a GMT bug we need to plot it from file otherwise the offset won't work
meca!("onemeca.dat", aki=(scale=0.6, font=(10,"Helvetica",:white)), fill=:black, offset=true)

# Add a little volcano symbol where an eruption was about to happen this year (2022) ... and it's not over yet.
plot!([-28 38.5], marker=(custom="volcano", size=0.4), ml=0.5, fill=:red, show=true)

### Change subject and let's look at tides

Did you know that the solid Earth also has tides? And they are not that small.

#### Compute the vertical component of the solid Earth tide for the day July 7 2022

In [ ]:
G = earthtide(T="2022-07-07T12:00:00");

In [ ]:
# Display it
imshow(G, coast=true, colorbar=true, title="Earth tide at noon 7-July-2022")

Let's look also at the individual components. For that we specify a location and a time period.
We will start be looking at one day worth of tides.

In [ ]:
D = earthtide(range="2022-07-07T/2022-07-08T/1m", location=(-82,9))
show(D)

And now we plot all three of them in the same figure.

In [ ]:
plot(D[:Time, :Vertical], lc=:red, lw=1, legend=:Vertical,
     title="Earth tide components (m) at Panama Cannal")
plot!(D[:Time, :East],  lc=:green, lw=1, legend=:East)
plot!(D[:Time, :North], lc=:blue,  lw=1, legend=:North, show=true)

What does it look like over one month?

In [ ]:
D = earthtide(range=("2022-07-01T", "2022-07-31T", "1m"), location=(-82,9));
plot(D[:Time, :Vertical], lc=:blue, legend=:Vertical, title="Tides (m), one month", show=true)